# HW7: Neural Transition-Based Dependency Parsing


In this exercise, you are going to build a deep learning model for Neural Networks Transition-Based Dependency Parsing. A dependency parser analyzes the grammatical structure of a sentence, establishing relationships between “head” words and words which modify those heads. Your implementation will be a transition-based parser, which incrementally builds up a parse one step at a time.

To complete this exercise, you will need to complete the code and build a deep learning model for dependency parsing. We will evaluate the model on the subset of Penn Treebank (annotated with Universal Dependencies). 

We provide the code for data preparation and the skeleton for PartialParse class. You do not need to understand the code outside of this notebook. 

This homework and the starter codes are adopt from Stanford University class CS224n.

This homework does not require you to use Google Colab or Google Cloud as the model is quite small (but you can still use it if you want)

## 1. Transition-Based Dependency Parsing

Your implementation will be a transition-based parser, which incrementally builds
up a parse one step at a time. At every step it maintains a partial parse, which is represented as follows:
- A stack of words that are currently being processed.
- A buffer of words yet to be processed.
- A list of dependencies predicted by the parser.

Initially, the stack only contains ROOT, the dependencies lists is empty, and the buffer contains all words
of the sentence in order. At each step, the parse applies a transition to the partial parse until its buffer is
empty and the stack is size 1. The following transitions can be applied:
- SHIFT: removes the first word from the buffer and pushes it onto the stack.
- LEFT-ARC: marks the second (second most recently added) item on the stack as a dependent of the
first item and removes the second item from the stack.
- RIGHT-ARC: marks the first (most recently added) item on the stack as a dependent of the second
item and removes the first item from the stack.

Your parser will decide among transitions at each state using a neural network classifier.

### TODO 1 (Written):
Go through the sequence of transitions needed for parsing the sentence “I parsed
this sentence correctly”. The dependency tree for the sentence is shown below. At each step, give the
configuration of the stack and buffer, as well as what transition was applied this step and what new
dependency was added (if any). The first three steps are provided below as an example.

<img src="https://github.com/ekapolc/NLP_2020/blob/master/HW7/images/img1.jpg?raw=true">

Complete the following table (double click the table and fill in the rest):

| stack    |  buffer |  new dependency | transition |
| :------: |:------: | :-------------: | :--------: |
| \[ROOT\]            | \[I, parsed, this, sentence, correctly\] |          | Initial Configuration |
| \[ROOT, I\]         | \[parsed, this, sentence, correctly\]    |          | SHIFT |
| \[ROOT, I, parsed\] | \[this, sentence, correctly\]            |          | SHIFT |
| \[ROOT, parsed\]    | \[this, sentence, correctly\]            | parsed→I | LEFT-ARC |
| \[ROOT, parsed , this\]  | \[sentence, correctly\]| |SHIFT |
| \[ROOT, parsed , this , sentence\]  | \[ correctly\]| |SHIFT |
| \[ROOT, parsed , sentence\]  | \[ correctly\]|sentence→this |LEFT-ARC |
| \[ROOT, parsed \]  | \[ correctly\]|parsed→sentence |RIGHT-ARC |
| \[ROOT, parsed , corectly \]  | \[\]| |SHIFT |
| \[ROOT, parsed \]  | \[\]| parsed→correctly |RIGHT-ARC|
| \[ROOT \]  | \[\]| ROOT→parsed |RIGHT-ARC|

### TODO 2 (Coding):
Implement the __\_\_init\_\___ and __parse_step__ functions in the PartialParse class. Your code must past both of the following tests.

In [ ]:
class PartialParse(object):
    def __init__(self, sentence):
        """Initializes this partial parse.

        Your code should initialize the following fields:
            self.stack: The current stack represented as a list with the top of the stack as the
                        last element of the list.
            self.buffer: The current buffer represented as a list with the first item on the
                         buffer as the first item of the list
            self.dependencies: The list of dependencies produced so far. Represented as a list of
                    tuples where each tuple is of the form (head, dependent).
                    Order for this list doesn't matter.

        The root token should be represented with the string "ROOT"

        Args:
            sentence: The sentence to be parsed as a list of words.
                      Your code should not modify the sentence.
        """
        # The sentence being parsed is kept for bookkeeping purposes. Do not use it in your code.
        self.sentence = sentence

        ### YOUR CODE HERE
        self.stack = ["ROOT"]
        self.buffer = list(sentence)
        self.dependencies = []
        ### END YOUR CODE

    def parse_step(self, transition):
        """Performs a single parse step by applying the given transition to this partial parse

        Args:
            transition: A string that equals "S", "LA", or "RA" representing the shift, left-arc,
                        and right-arc transitions. You can assume the provided transition is a legal
                        transition.
        """
        ### YOUR CODE HERE
        if transition == 'S' and len(self.buffer) > 0:
            word = self.buffer.pop(0)
            self.stack.append(word)
        if transition == 'LA' :
            head = self.stack[-1]
            dependent = self.stack.pop(-2)
            self.dependencies.append((head, dependent))
        if transition == 'RA':
            head = self.stack[-2]
            dependent = self.stack.pop()
            self.dependencies.append((head, dependent))
        ### END YOUR CODE

    def parse(self, transitions):
        """Applies the provided transitions to this PartialParse

        Args:
            transitions: The list of transitions in the order they should be applied
        Returns:
            dependencies: The list of dependencies produced when parsing the sentence. Represented
                          as a list of tuples where each tuple is of the form (head, dependent)
        """
        for transition in transitions:
            self.parse_step(transition)
        return self.dependencies

In [ ]:
# Do not modify this code
def test_step(name, transition, stack, buf, deps,
              ex_stack, ex_buf, ex_deps):
    """Tests that a single parse step returns the expected output"""
    pp = PartialParse([])
    pp.stack, pp.buffer, pp.dependencies = stack, buf, deps

    pp.parse_step(transition)
    stack, buf, deps = (tuple(pp.stack), tuple(pp.buffer), tuple(sorted(pp.dependencies)))
    assert stack == ex_stack, \
        "{:} test resulted in stack {:}, expected {:}".format(name, stack, ex_stack)
    assert buf == ex_buf, \
        "{:} test resulted in buffer {:}, expected {:}".format(name, buf, ex_buf)
    assert deps == ex_deps, \
        "{:} test resulted in dependency list {:}, expected {:}".format(name, deps, ex_deps)
    print("{:} test passed!".format(name))


def test_parse_step():
    """Simple tests for the PartialParse.parse_step function
    Warning: these are not exhaustive
    """
    test_step("SHIFT", "S", ["ROOT", "the"], ["cat", "sat"], [],
              ("ROOT", "the", "cat"), ("sat",), ())
    test_step("LEFT-ARC", "LA", ["ROOT", "the", "cat"], ["sat"], [],
              ("ROOT", "cat",), ("sat",), (("cat", "the"),))
    test_step("RIGHT-ARC", "RA", ["ROOT", "run", "fast"], [], [],
              ("ROOT", "run",), (), (("run", "fast"),))


def test_parse():
    """Simple tests for the PartialParse.parse function
    Warning: these are not exhaustive
    """
    sentence = ["parse", "this", "sentence"]
    dependencies = PartialParse(sentence).parse(["S", "S", "S", "LA", "RA", "RA"])
    dependencies = tuple(sorted(dependencies))
    expected = (('ROOT', 'parse'), ('parse', 'sentence'), ('sentence', 'this'))
    assert dependencies == expected,  \
        "parse test resulted in dependencies {:}, expected {:}".format(dependencies, expected)
    assert tuple(sentence) == ("parse", "this", "sentence"), \
        "parse test failed: the input sentence should not be modified"
    print("parse test passed!")

In [ ]:
test_parse_step()
test_parse()

SHIFT test passed!
LEFT-ARC test passed!
RIGHT-ARC test passed!
parse test passed!


### TODO 3 (Coding):
Our network will predict which transition should be applied next to a partial parse. We could use it to parse a single sentence by applying predicted transitions until the parse is complete. However, neural networks run much more efficiently when making predictions about batches of data at a time (i.e., predicting the next transition for a many different partial parses simultaneously). We can parse sentences in minibatches with the following algorithm.

<img src="https://github.com/ekapolc/NLP_2020/blob/master/HW7/images/img2.jpg?raw=true">

Implement this algorithm in the minibatch parse function and run the following test.

In [ ]:
def minibatch_parse(sentences, model, batch_size):
    """Parses a list of sentences in minibatches using a model.

    Args:
        sentences: A list of sentences to be parsed (each sentence is a list of words)
        model: The model that makes parsing decisions. It is assumed to have a function
               model.predict(partial_parses) that takes in a list of PartialParses as input and
               returns a list of transitions predicted for each parse. That is, after calling
                   transitions = model.predict(partial_parses)
               transitions[i] will be the next transition to apply to partial_parses[i].
        batch_size: The number of PartialParses to include in each minibatch
    Returns:
        dependencies: A list where each element is the dependencies list for a parsed sentence.
                      Ordering should be the same as in sentences (i.e., dependencies[i] should
                      contain the parse for sentences[i]).
    """

    ### YOUR CODE HERE
    num_sentences     = len(sentences)
    partial_parses    = [PartialParse(sentence) for sentence in sentences]
    unfinished_parses = partial_parses[:] # shallow copy of pp

    while len(unfinished_parses) > 0:
        parsers = unfinished_parses[:batch_size]
        batch_transitions = model.predict(parsers)
        for i in range(len(parsers)):
            parsers[i].parse([batch_transitions[i]])
            if len(parsers[i].buffer) == 0 and len(parsers[i].stack) == 1:
                unfinished_parses.remove(parsers[i])

    dependencies = [pp.dependencies for pp in partial_parses]
    ### END YOUR CODE

    return dependencies

In [ ]:
# Do not modify this code
class DummyModel(object):
    """Dummy model for testing the minibatch_parse function
    First shifts everything onto the stack and then does exclusively right arcs if the first word of
    the sentence is "right", "left" if otherwise.
    """
    def predict(self, partial_parses):
        return [("RA" if pp.stack[1] is "right" else "LA") if len(pp.buffer) == 0 else "S"
                for pp in partial_parses]


def test_dependencies(name, deps, ex_deps):
    """Tests the provided dependencies match the expected dependencies"""
    deps = tuple(sorted(deps))
    assert deps == ex_deps, \
        "{:} test resulted in dependency list {:}, expected {:}".format(name, deps, ex_deps)


def test_minibatch_parse():
    """Simple tests for the minibatch_parse function
    Warning: these are not exhaustive
    """
    sentences = [["right", "arcs", "only"],
                 ["right", "arcs", "only", "again"],
                 ["left", "arcs", "only"],
                 ["left", "arcs", "only", "again"]]
    deps = minibatch_parse(sentences, DummyModel(), 2)
    test_dependencies("minibatch_parse", deps[0],
                      (('ROOT', 'right'), ('arcs', 'only'), ('right', 'arcs')))
    test_dependencies("minibatch_parse", deps[1],
                      (('ROOT', 'right'), ('arcs', 'only'), ('only', 'again'), ('right', 'arcs')))
    test_dependencies("minibatch_parse", deps[2],
                      (('only', 'ROOT'), ('only', 'arcs'), ('only', 'left')))
    test_dependencies("minibatch_parse", deps[3],
                      (('again', 'ROOT'), ('again', 'arcs'), ('again', 'left'), ('again', 'only')))
    print("minibatch_parse test passed!")

In [ ]:
test_minibatch_parse()

minibatch_parse test passed!


## 2. Setup and Preprocessing

In [ ]:
!unzip HW7_parsing.zip -d /content


Archive:  HW7_parsing.zip
  inflating: /content/data/dev.conll  
  inflating: /content/data/dev.gold.conll  
  inflating: /content/data/en-cw.txt  
  inflating: /content/data/test.conll  
  inflating: /content/data/test.gold.conll  
  inflating: /content/data/train.conll  
  inflating: /content/data/train.gold.conll  
  inflating: /content/HW7_parsing.ipynb  
  inflating: /content/images/.ipynb_checkpoints/img1-checkpoint.jpg  
  inflating: /content/images/img1.jpg  
  inflating: /content/images/img2.jpg  
  inflating: /content/README.md      
  inflating: /content/utils/.ipynb_checkpoints/general_utils-checkpoint.py  
  inflating: /content/utils/.ipynb_checkpoints/parser_utils-checkpoint.py  
 extracting: /content/utils/__init__.py  
  inflating: /content/utils/__pycache__/__init__.cpython-35.pyc  
  inflating: /content/utils/__pycache__/general_utils.cpython-35.pyc  
  inflating: /content/utils/__pycache__/parser_utils.cpython-35.pyc  
  inflating: /content/utils/general_utils.py  
 

In [ ]:
from utils.parser_utils import minibatches, load_and_preprocess_data

Preparing data. We will use a subset of Penn Treebank and pretrained embeddings in this task

We are now going to train a neural network to predict, given the state of the stack, buffer, and dependencies, which transition should be applied next. First, the model extracts a feature vector representing the current state. We will be using the feature set presented in the original neural dependency parsing paper: A Fast and Accurate Dependency Parser using Neural Networks. 

The function extracting these features has been implemented for you in parser_utils. This feature vector consists of a list of tokens (e.g., the last word in the stack, first word in the buffer, dependent of the second-to-last word in the stack if there is one, etc.). They can be represented as a list of integers.

In [ ]:
parser, embeddings, train_examples, dev_set, test_set = load_and_preprocess_data(True)

Loading data...
took 2.36 seconds
Building parser...
took 0.03 seconds
Loading pretrained embeddings...
took 2.38 seconds
Vectorizing data...
took 0.07 seconds
Preprocessing training data...
took 1.39 seconds


In [ ]:
print(len(train_examples), len(dev_set), len(test_set))

48390 500 500


In [ ]:
print(embeddings.shape)

(5157, 50)


Get the full batch of our subset data

In [ ]:
minibatch_gen = minibatches(train_examples, len(train_examples))
x_train, y_train = minibatch_gen.__next__()

In [ ]:
print(x_train.shape)
print(y_train.shape)

(48390, 36)
(48390, 3)


In [ ]:
# Sample features
print(x_train[10:])

[[5156  134 5054 ...   83   83   83]
 [1093  250  714 ...   83   83   83]
 [  94 2407 1435 ...   83   83   83]
 ...
 [4263  580   98 ...   83   83   83]
 [5155 5156 1257 ...   83   83   83]
 [ 243  112  262 ...   45   83   83]]


In [ ]:
ma = 0
for x in x_train:
  for i in x:
    ma = max(ma,i)
for d in dev_set:
  for w in d["word"]:
    ma = max(ma,w)
for d in test_set:
  for w in d["word"]:
    ma = max(ma,w)
print(ma)

5156


In [ ]:
print(y_train[:10])

[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [ ]:
print(dev_set[0])
print(test_set[0])

{'word': [5156, 5154, 524, 88, 85, 212, 2077, 92, 715, 821, 843, 2207, 95, 122, 5154, 241, 85, 119, 4617, 655, 215, 181, 425, 350, 86, 1114, 222, 1958, 3213, 89, 85, 168, 93, 397, 88, 2079, 2080, 87], 'pos': [84, 43, 44, 40, 41, 42, 66, 51, 42, 42, 48, 39, 64, 58, 50, 70, 41, 43, 39, 39, 39, 58, 50, 39, 45, 56, 41, 43, 39, 52, 41, 39, 60, 39, 40, 43, 44, 46], 'head': [-1, 2, 10, 6, 6, 6, 2, 6, 9, 6, 0, 10, 14, 14, 11, 22, 20, 20, 20, 20, 22, 22, 14, 22, 14, 14, 28, 28, 25, 33, 31, 33, 31, 28, 36, 36, 33, 10], 'label': [-1, 19, 2, 35, 29, 18, 13, 20, 18, 21, 0, 17, 2, 32, 34, 8, 29, 19, 18, 18, 2, 32, 12, 17, 6, 31, 29, 19, 17, 35, 29, 7, 35, 13, 35, 19, 13, 6]}
{'word': [5156, 176, 86, 101, 103, 118, 841, 391, 87], 'pos': [84, 41, 45, 54, 48, 47, 43, 42, 46], 'head': [-1, 7, 7, 7, 7, 7, 7, 0, 7], 'label': [-1, 9, 6, 2, 36, 25, 18, 0, 6]}


## 3. Model

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input, Dense, Reshape, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

### TODO 4 (Coding):
Build and train a tensroflow keras model to predict an action for each state of of the input. This is a simple classification task. 
- The input and output of the model must match the dimention of x_train and y_train.
- The model must use the provided pretrained embeddings
- The model could comprise of only a feedforward layer and a dropout
- Training loss should be around 0.1 or below, and training categorical_accuracy above 0.94

In [ ]:
model = Sequential()
# Write your code here
inputs = Input(shape=(36,), dtype='int32')
x = (Embedding(5157 ,50,weights=[embeddings],input_length=36,trainable=True,mask_zero=True))(inputs)
x = Flatten()(x)
x = Dense(900, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(450, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(3, activation='softmax')(x)
model = Model(inputs, output)
model.compile(optimizer=Adam(lr=0.001),  loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()
pass

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 36)]              0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 36, 50)            257850    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1800)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 900)               1620900   
_________________________________________________________________
dropout_4 (Dropout)          (None, 900)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 450)               405450    
_________________________________________________________________
dropout_5 (Dropout)          (None, 450)               0   

In [ ]:
# Write your code here
model.fit(x_train,y=y_train, batch_size=64,epochs=10,verbose=1)

Epoch 1/10
757/757 [==============================] - 26s 34ms/step - loss: 0.4943 - categorical_accuracy: 0.8196
Epoch 2/10
757/757 [==============================] - 27s 36ms/step - loss: 0.1867 - categorical_accuracy: 0.9277
Epoch 3/10
757/757 [==============================] - 27s 36ms/step - loss: 0.1259 - categorical_accuracy: 0.9507
Epoch 4/10
757/757 [==============================] - 27s 35ms/step - loss: 0.0977 - categorical_accuracy: 0.9636
Epoch 5/10
757/757 [==============================] - 25s 33ms/step - loss: 0.0719 - categorical_accuracy: 0.9732
Epoch 6/10
757/757 [==============================] - 25s 33ms/step - loss: 0.0514 - categorical_accuracy: 0.9810
Epoch 7/10
757/757 [==============================] - 25s 33ms/step - loss: 0.0415 - categorical_accuracy: 0.9843
Epoch 8/10
757/757 [==============================] - 25s 33ms/step - loss: 0.0335 - categorical_accuracy: 0.9884
Epoch 9/10
757/757 [==============================] - 25s 34ms/step - loss: 0.0241 - cat

## 4. Evaluation

For Dependency Parsing, we usually report attachment score of the model for evaluation. There are two possible metrics UAS and LAS.

### TODO 5 (Written and Coding):
Explain how attachment score is calculated and the difference between unlabeled attachment score (UAS) and labeled attachment score (LAS). Which one should we use here?

__Answer here__:

In calculation of accuracy we user the formula :    **ACCURACY = number of correct dependecies from prediction / number of all dependencies**

In UAS, it only checks about correction of dependencies line, but in LAS, it also check that the label of both dependencies are correct.

By that reason, **In this model it can only use UAS because we only have predict dependencies.**

Report the score using appropriate metric on dev_set and test_set. The function for calculating scores are provided in parser.parse and the dataset can be passed in as-is.

dev_score:

In [ ]:
dev_UAS ,dev_dependencies_pred = parser.parse(dev_set,model,minibatch_parse)
pass

In [ ]:
print("UAS score is :",dev_UAS*100,"%")
#print(dev_dependencies_pred[0])

UAS score is : 77.97217362326086 %


test_score:

In [ ]:
test_UAS ,test_dependencies_pred = parser.parse(test_set,model,minibatch_parse)
pass

In [ ]:
print("UAS score is :",test_UAS*100,"%")
#print(test_dependencies_pred[0])

UAS score is : 78.62910008410428 %


Also, print one sample sentence (in English) in the test set and its predicted dependencies from the model.
You can use __parser.id2tok\[word_id\]__ to lookup the word in English.

__Draw a picture of this sentence with arrows and upload it to my couseville__

In [ ]:
import random
rta = random.randint(0, 100) 
pass

In [ ]:
rta

76

In [ ]:
print("  ".join([ parser.id2tok[word_id]+":"+str(i) for i,word_id in enumerate(test_set[rta]["word"])]))

<ROOT>:0  <UNK>:1  futures:2  contracts:3  <UNK>:4  at:5  much:6  lower:7  prices:8  than:9  <UNK>:10  of:11  the:12  stock:13  market:14  itself:15  .:16


In [ ]:
print(test_dependencies_pred[rta])

[(3, 2), (3, 1), (4, 3), (7, 6), (8, 7), (8, 5), (10, 9), (14, 13), (14, 12), (14, 11), (14, 15), (10, 14), (8, 10), (4, 8), (4, 16), (0, 4)]


In [ ]:
depen = {}
for a in test_dependencies_pred[rta]:
  if a[0] not in depen:
    depen[a[0]]=[a[1]]
  else:
    depen[a[0]].append(a[1])
print(depen)

{3: [2, 1], 4: [3, 8, 16], 7: [6], 8: [7, 5, 10], 10: [9, 14], 14: [13, 12, 11, 15], 0: [4]}
